In [1]:
import cftime
import glob
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
from scipy import stats

import mission_tools

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
mission_list = glob.glob('../microSWIFT_data/final_dataset/mission_*.nc')

hs_all = []
tm_all = []
mean_dir_all = []
xs_gamma_loc_all = []
water_level_all = []
mission_number = []
mission_break_depth_all = []
mission_time_all = []
mission_spectra_all = []
mission_dir_spectra_all = []
freq_all = []
dir_bins_all = []

gamma = 0.35
awac_file = '../microSWIFT_data/FRFdata/FRF-ocean_waves_awac-4.5m_202110.nc'
bathy_file = '../microSWIFT_data/FRFdata/FRF_geomorphology_DEMs_surveyDEM_20211021.nc'
bathy_dataset = nc.Dataset(bathy_file)
xFRF_grid, yFRF_grid = np.meshgrid(bathy_dataset['xFRF'][:],bathy_dataset['yFRF'][:])
bathy = bathy_dataset['elevation'][0,:,:]
waterlevel_file = '../microSWIFT_data/FRFdata/FRF-ocean_waterlevel_eopNoaaTide_202110.nc'
waterlevel_dataset = nc.Dataset(waterlevel_file)
waterlevel = waterlevel_dataset['waterLevel'][:]
waterlevel_time = waterlevel_dataset['time'][:]

for mission_nc in mission_list:
    mission_dataset = nc.Dataset(mission_nc, mode='r')

    mission_time = np.median(mission_dataset['time'][:])

    # Compute depth that waves should be breaking in 
    mission_hs = mission_tools.closest_awac_sig_wave_height(mission_time, awac_file)
    mission_tm = mission_tools.closest_awac_Tm(mission_time, awac_file)
    mission_mean_dir = mission_tools.closest_awac_dir(mission_time, awac_file)
    mission_spectra, freq = mission_tools.closest_awac_spectra(mission_time, awac_file)
    mission_dir_spectra, _, dirs = mission_tools.closest_awac_dir_spectra(mission_time, awac_file)

    # Compute 
    mission_break_depth = mission_hs / gamma
    mission_xs_gamma_location = np.interp(-mission_break_depth, np.flip(np.nanmean(bathy, axis=0)), np.flip(bathy_dataset['xFRF'][:]))

    # Save all values
    hs_all.append(mission_hs)
    tm_all.append(mission_tm)
    mean_dir_all.append(mission_mean_dir)
    xs_gamma_loc_all.append(mission_xs_gamma_location)
    mission_break_depth_all.append(mission_break_depth)
    mission_time_all.append(mission_time)
    mission_spectra_all.append(mission_spectra.tolist())
    freq_all.append(freq.tolist())
    dir_bins_all.append(dirs.tolist())
    mission_dir_spectra_all.append(mission_dir_spectra.tolist())
    
    # Save mission number
    mission_number.append(int(mission_nc[41:-3]))

/opt/homebrew/Caskroom/mambaforge/base/envs/dunex-venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:784: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [3]:
mission_df = pd.DataFrame(cftime.num2pydate(mission_time_all, units=mission_dataset['time'].units,
                          calendar=mission_dataset['time'].calendar), columns=['time'])
mission_df['mission number'] = mission_number
mission_df['Hs [m]'] = hs_all
mission_df['Tm [s]'] = tm_all
mission_df['Mean Dir [deg]'] = mean_dir_all
mission_df['cross shore gamma location [m]'] = xs_gamma_loc_all
mission_df['break depth'] = mission_break_depth_all
mission_df['freq [hz]'] = freq_all
mission_df['energy density [m^2\hz]'] = mission_spectra_all
mission_df['direction bins [deg]'] = dir_bins_all
mission_df['directional energy density [m^2/hz/deg]'] = mission_dir_spectra_all

mission_df

,time,mission number,Hs [m],Tm [s],Mean Dir [deg],cross shore gamma location [m],break depth,freq [hz],energy density [m^2\hz],direction bins [deg],directional energy density [m^2/hz/deg]
0,2021-10-29 14:33:54.998780,78,2.313170,5.764382,80.071942,579.607826,6.609056,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.15948983688741117, 0.1819984354582862, 0.38...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[0.00034500446054153144, 0.000364556792192161..."
1,2021-10-13 13:19:59.997600,29,1.236368,5.767854,65.434668,291.894318,3.532481,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.04970083720291576, 0.041036184162413815, 0....","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[0.0001457403413951397, 0.0001440187334083020..."
2,2021-10-07 14:34:59.998800,9,1.310994,4.890865,68.621922,318.032279,3.745698,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.028828260370244017, 0.027199895525149783, 0...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[6.295069033512846e-05, 6.452057277783751e-05..."
3,2021-10-15 16:19:29.998200,39,0.663709,7.104419,82.901678,212.502257,1.896310,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.013718874925943907, 0.017029714357263682, 0...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[3.791501512750983e-05, 3.852572990581393e-05..."
4,2021-10-28 15:41:59.998080,68,1.659354,6.439816,64.004803,422.389669,4.741013,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.11050346628957844, 0.10613922829665084, 0.1...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[0.00029926502611488104, 0.000300445535685867..."
...,...,...,...,...,...,...,...,...,...,...,...
62,2021-10-29 13:05:59.999280,75,2.518600,5.855118,79.555679,649.353347,7.196001,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.1672283125137516, 0.2068343427800254, 0.367...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[0.00033313207677565515, 0.000367225060472264..."
63,2021-10-11 13:21:29.997960,24,2.385967,5.394237,60.142094,602.358868,6.817050,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.15125480450146073, 0.19276349687774344, 0.2...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[0.00034638005308806896, 0.000354821211658418..."
64,2021-10-05 18:18:29.998680,4,0.606500,8.126498,79.887002,203.788744,1.732858,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.017535387845646812, 0.016989574288081014, 0...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[4.0331789932679385e-05, 4.031559728900902e-0..."
65,2021-10-14 13:07:29.998200,34,0.923776,6.564444,75.851086,243.056756,2.639359,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.031141506778260747, 0.023073153485875318, 0...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[6.985702930251136e-05, 7.242430001497269e-05..."


In [4]:
mission_df.to_csv('../microSWIFT_data/mission_df_45mawac.csv')